# EXTRACT

In [221]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import json

my_token = "BQDybnuzjuhdsxm3JX1UW81chsIfdQSu0g_9Jq47PxKt5YxJ-eLkDK0b2qkTAUCI5PvcMCvLssmH1wVIH415Itvn9PTegt8Aa9LLnWHEi4BhwVtHwccEcvnqfwovnXaepqOvJTTR7LVGrtWFH9mZoAc_2UdWOO0uCg-pQEC7mFhXYo4Bcrk"

today_datetime = datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1)
yesterday_timestamp_milliseconds = int(yesterday_datetime.timestamp()) * 1000
yesterday_timestamp_milliseconds = int(yesterday_datetime.timestamp()) * 1000

headers = {
    "Accept" : "application/json",
    "Content-Type" : "application/json",
    "Authorization" : "Bearer {token}".format(token=my_token)
}

r = requests.get("https://api.spotify.com/v1/me/player/recently-played?before={time}".format(time=yesterday_timestamp_milliseconds), headers = headers)
data = r.json()
print(data)

#print(data["items"])

{'error': {'status': 401, 'message': 'The access token expired'}}


In [220]:
artist_names = []
song_names = []

for songs in data["items"]:
    #artist_names.append(songs["id"]["name"])
    song_names.append(songs["track"]["name"])

artist_names

KeyError: 'items'

In [222]:
for i in data["items"] :
    print(i["track"])

KeyError: 'items'